<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C2-Writing_A_Sniffer/Host_Scanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Decoding ICMP__*

Now that we can fully decode the IP layer of any sniffed packets, we have to be able to decode the __ICMP__ responses that our scanner will elicit from sending __UDP__ datagrams to closed ports. ICMP messages can vary greatly in their contents, but each message contains three elements that stay consistent: The type, code, and checksum fields. The type and code fields tell the receiving host what type of ICMP message is arriving, which then dictates how to decode it properly.

For the purpose of our scanner, we are looking for a type value of 3 and a code value of 3. This corresponds to the __Destination Unreachable__ class of ICMP messages, and the code value of 3 indicates that the __Port Unreachable__ error has been caused. Refer to __Figure 3-3__ for a diagram of a Destination Unreachable ICMP message.

<div align="center" width="100%">
<img src="https://github.com/damianiRiccardo90/BHP/blob/master/C2-Writing_A_Sniffer/ICMP_Destination_Unreachable_Message.png?raw=true" alt="From Client to Server" width="50%">
<p style="text-align:center"><em><strong>Figure 3-3:</strong> Diagram of Destination Unreachable ICMP message</em></p>
</div>

As you can see, the first 8 bits are the type, and the second 8 bits contain our ICMP code. One interesting thing to note is that when a host sends one of these ICMP messages, it actually includes the IP header of the originating message that generated the response. We can also see that we will double-check against 8 bytes of the original datagram that was sent in order to make sure our scanner generated the ICMP response. To do so, we simply slice off the last 8 bytes of the received buffer to pull out the magic string that our scanner sends.

Let's add some more code to our previous sniffer to inlcude the ability to decode ICMP packets. Let's save our previous file as __sniffer_with_icmp.py__ and add the following code:

In [ ]:
import ipaddress
import os
import socket
import struct
import sys

### CODE FROM PREVIOUS PARAGRAPH ###
class IP:
    def __init__(self, buff=None):
        header = struct.unpack('<' + "BBHHHBBH4s4s", buff)
        self.ver = header[0] >> 4
        self.idl = header[0] & 0xF

        self.tos = header[1]
        self.len = header[2]
        self.id = header[3]
        self.offset = header[4]
        self.ttl = header[5]
        self.protocol_num = header[6]
        self.sum = header[7]
        self.src = header[8]
        self.dst = header[9]

        # Human readable IP addresses
        self.src_address = ipaddress.ip_address(self.src)
        self.dst_address = ipaddress.ip_address(self.dst)

        # Map protocol constants to their names
        self.protocol_map = {1: "ICMP", 6: "TCP", 17: "UDP"}
        try:
            self.protocol = self.protocol_map[self.protocol_num]
        except Exception as e:
            print("%s No protocol for %s" % (e, self.protocol_num))
            self.protocol = str(self.protocol_num)
####################################

class ICMP:
    def __init__(self, buff):
        header = struct.unpack("<" + "BBHHH", buff)
        self.type = header[0]
        self.code = header[1]
        self.sum = header[2]
        self.id = header[3]
        self.seq = header[4]

def sniff(host):
    ### CODE FROM PREVIOUS PARAGRAPH ###
    if os.name == "nt":
        socket_protocol = socket.IPPROTO_IP
    else:
        socket_protocol = socket.IPPROTO_ICMP

    sniffer = socket.socket(socket.AF_INET, socket.SOCK_RAW, socket_protocol)
    sniffer.bind((host, 0))
    sniffer.setsockopt(socket.IPPROTO_IP, socket.IP_HDRINCL, 1)

    if os.name == "nt":
        sniffer.ioctl(socket.STO_RCVALL, socket.RCVALL_ON)

    try:
        while True:
            raw_buffer = sniffer.recvfrom(65535)[0]
            ip_header = IP(raw_buffer[0:20])
            print("Protocol: %s %s -> %s" % (ip_header.protocol, 
                                             ip_header.src_address, 
                                             ip_header.dst_address))
    except KeyboardInterrupt:
        if os.name == "nt":
            sniffer.ioctl(socket.SIO:RCVALL, socket.RCVALL_OFF)
        sys.exit()
    ####################################

    